In [1]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'C:/Users/grayni/Desktop/tr_py/SQL/bigquery-362520-0ed0e93eb083.json'

In [2]:
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql.ex5 import *
print("Setup Complete")

Setup Complete


In [3]:
from google.cloud import bigquery

client = bigquery.Client()

dataset_ref = client.dataset('chicago_taxi_trips', project='bigquery-public-data')

dataset = client.get_dataset(dataset_ref)

In [4]:
tables = client.list_tables(dataset)

In [5]:
for table in tables:
    print(table.table_id)

taxi_trips


In [6]:
table_name = 'taxi_trips'

q_1.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [7]:
table_ref = dataset_ref.table('taxi_trips')
table = client.get_table(table_ref)

client.list_rows(table, max_results=5).to_dataframe()

,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,839771e41537432c54c7099e8ea1bf43887ca484,92195ba6d79380aa9da1f2db98ac05a3eff859c4151f24...,2015-05-01 18:00:00+00:00,2015-05-01 18:00:00+00:00,0,0.0,<NA>,<NA>,<NA>,<NA>,...,0.0,11.82,Credit Card,Chicago Elite Cab Corp. (Chicago Carriag,NaN,NaN,None,NaN,NaN,None
1,9437e8feb58e5cd3cb22a9bb91cbcf3bc60ee9a4,3e0ecc411309f519aa2278ee885227cde56891acadc652...,2015-05-03 11:15:00+00:00,2015-05-03 11:15:00+00:00,0,0.0,<NA>,<NA>,<NA>,<NA>,...,0.0,13.85,Credit Card,Suburban Dispatch LLC,NaN,NaN,None,NaN,NaN,None
2,8162b1f82324cf1270b4d1d7a5498ea5150501cf,36dee7e88e2048dfeb8a98c9e420a1600544a8d4f21fe6...,2015-05-09 01:15:00+00:00,2015-05-09 01:15:00+00:00,0,0.0,<NA>,<NA>,<NA>,<NA>,...,0.0,20.22,Credit Card,Chicago Elite Cab Corp. (Chicago Carriag,NaN,NaN,None,NaN,NaN,None
3,13f8cff0ef83a0e6feed5f67193e5b7e705eb7ab,35f259e52dda4ae7ef17bbcf2f61be54445372dc471929...,2015-04-14 14:45:00+00:00,2015-04-14 14:45:00+00:00,0,0.0,<NA>,<NA>,<NA>,<NA>,...,0.0,6.85,Credit Card,Chicago Elite Cab Corp. (Chicago Carriag,NaN,NaN,None,NaN,NaN,None
4,f4586d5fba35a944615fc6e75fc7c57a69cf9fa8,5b54ed6a93f455a3a35cec10d5708caa67ac3dfaf6727a...,2015-08-26 11:00:00+00:00,2015-08-26 11:15:00+00:00,540,1.7,<NA>,<NA>,<NA>,<NA>,...,0.0,7.45,Cash,None,NaN,NaN,None,NaN,NaN,None


In [8]:
rides_per_year_query = """
                       SELECT EXTRACT(YEAR FROM trip_start_timestamp) AS year, COUNT(1) AS num_trips
                       FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                       GROUP BY year
                       ORDER BY year
                       """

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
rides_per_year_query_job = client.query(rides_per_year_query, job_config=safe_config)

rides_per_year_result = rides_per_year_query_job.to_dataframe()
print(rides_per_year_result)
#q_3.check()

   year  num_trips
0  2013   27217716
1  2014   37395436
2  2015   32385875
3  2016   31759339
4  2017   24988003
5  2018   20732088
6  2019   16477365
7  2020    3889032
8  2021    3948045
9  2022    4110845


In [18]:
rides_per_month_query = """
                        SELECT EXTRACT(MONTH FROM trip_start_timestamp) AS month, 
                               COUNT(1) AS num_trips
                        FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                        WHERE EXTRACT(YEAR FROM trip_start_timestamp) = 2017
                        GROUP BY month
                        ORDER BY month
                        """

# Set up the query (cancel the query if it would use too much of 
# your quota)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
rides_per_month_query_job = client.query(rides_per_month_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
rides_per_month_result = rides_per_month_query_job.to_dataframe()

q_4.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [24]:
speeds_query = """
               WITH RelevantRides AS
               (
                   SELECT EXTRACT(HOUR FROM trip_start_timestamp) AS hour_of_day, trip_miles, trip_seconds
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE trip_start_timestamp > '2017-01-01' AND 
                         trip_start_timestamp < '2017-07-01' AND
                         trip_seconds > 0 AND
                         trip_miles > 0
               )
               SELECT hour_of_day,
                      COUNT(1) AS num_trips,
                      3600 * SUM(trip_miles) / SUM(trip_seconds) AS avg_mph
               FROM RelevantRides
               GROUP BY hour_of_day
               ORDER BY hour_of_day
               """

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
speeds_query_job = client.query(speeds_query, job_config=safe_config)

speeds_result = speeds_query_job.to_dataframe()

# View results
print(speeds_result)

# Check your answer
q_5.check()

    hour_of_day  num_trips    avg_mph
0             0     319339  20.230524
1             1     266529  18.937621
2             2     210147  18.777070
3             3     159668  20.158048
4             4     122183  26.736014
5             5     119312  30.769172
6             6     182738  24.588313
7             7     358406  17.735967
8             8     541775  15.079892
9             9     565548  16.543882
10           10     525120  18.539614
11           11     594603  18.928379
12           12     622324  17.838745
13           13     630181  17.671089
14           14     622465  16.974239
15           15     640430  15.688418
16           16     701435  14.283888
17           17     756627  12.462955
18           18     768251  13.646810
19           19     701064  16.642882
20           20     598614  19.536777
21           21     552726  20.433874
22           22     501095  19.531374
23           23     399587  19.877046


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>